In [1]:
from lsqecc.logical_lattice_ops import logical_lattice_ops
from lsqecc.patches.patches import PatchType
from lsqecc.patches.ancilla_region_routing import make_graph_of_free_cells, make_tuple, AncillaRegionRoutingException
from lsqecc.pipeline.lattice_surgery_compilation_pipeline import compile_str
from lsqecc.simulation.logical_patch_state_simulation import SimulatorType

/Users/jviszlai/Documents/Work/Research/ProjectRepos/swiper/.venv/lib/python3.10/site-packages/cotengra/hyperoptimizers/hyper.py:33: UserWarning: Couldn't import `kahypar` - skipping from default hyper optimizer and using basic `labels` method instead.
  warnings.warn(


#### pyLIQTR Carleman Encoding

In [2]:
import math
import numpy as np
from qualtran._infra.gate_with_registers import get_named_qubits
from qualtran import Register, Signature, QAny

from pyLIQTR.BlockEncodings.CarlemanLinearization   import Carleman_Linearization
from pyLIQTR.ProblemInstances.NonlinearODE          import FOperators
from pyLIQTR.utils.circuit_decomposition            import circuit_decompose_multi

import cirq

In [3]:
n = 2
K = 4

a0_in = n
a1 = n
a2_in = math.ceil(n/2)
a2_out = math.ceil(n)

alpha0 = 1
alpha1 = 1
alpha2 = 1

In [4]:
ancilla_register = Register("ancilla", QAny(bitsize = 7 + max(a0_in, a1, a2_in, a2_out) + math.ceil(np.log2(K))))
data_register = Register("data", QAny(bitsize = n*K+1))

signature = Signature([ancilla_register, data_register])
registers = get_named_qubits(signature)

In [5]:
carlemanLinearization = Carleman_Linearization(FOperators(n, K, (a0_in, a1, a2_in, a2_out), (alpha0, alpha1, alpha2)), K)
circuit = cirq.Circuit(carlemanLinearization.on_registers(**registers))


### Lattice Surgery Schedule

In [6]:
from benchmarks.cirq_to_ls import cirq_to_ls
from swiper2.simulator import DecodingSimulator
import matplotlib.pyplot as plt
import swiper2.plot as plotter

In [7]:
circuit_decompose_multi(circuit, 6)[:2]

gancilla_c(0): ───H──────────T───

gancilla_c(1): ───H──────────T───

gancilla_c(2): ───H──────────T───

gancilla_c(3): ───H──────────T───

gancilla_c(4): ───H──────────T───

gancilla_c(5): ───H──────────T───

gancilla_c(6): ───H──────────T───

gancilla_c(7): ───H──────────T───

ancilla0: ────────Ry(0.5π)───────

ancilla1: ────────Ry(0.5π)───────

ancilla2: ────────X──────────────

ancilla7: ────────H──────────────

ancilla8: ────────H──────────────

In [8]:
ls_schedule = cirq_to_ls(circuit_decompose_multi(circuit, 6)[:2])

LS Instructions read  107
Slices 9
Made patch computation. Took 0.00528825s.
LS Instructions read  107
Slices 9
Made patch computation. Took 0.0122405s.


In [9]:
ls_schedule.all_instructions

[Instruction(name='INJECT_T', idx=0, patches=frozenset({(0, 2)}), duration=<Duration.D: 1>, conditioned_on_idx=frozenset(), conditional_dependencies=frozenset(), conditioned_on_completion_idx=frozenset(), conditional_completion_dependencies=frozenset(), merge_faces=frozenset()),
 Instruction(name='MERGE', idx=1, patches=frozenset({(0, 2), (1, 2), (2, 2)}), duration=<Duration.D: 1>, conditioned_on_idx=frozenset(), conditional_dependencies=frozenset(), conditioned_on_completion_idx=frozenset(), conditional_completion_dependencies=frozenset(), merge_faces=frozenset({((0, 2), (1, 2)), ((1, 2), (2, 2))})),
 Instruction(name='DISCARD', idx=2, patches=frozenset({(1, 2)}), duration=0, conditioned_on_idx=frozenset(), conditional_dependencies=frozenset({40}), conditioned_on_completion_idx=frozenset(), conditional_completion_dependencies=frozenset(), merge_faces=frozenset()),
 Instruction(name='DISCARD', idx=3, patches=frozenset({(0, 2)}), duration=0, conditioned_on_idx=frozenset(), conditional_d

In [10]:
d=7
decoding_time = 0
speculation_time = 0
speculation_accuracy = 1

simulator = DecodingSimulator(d, lambda _: decoding_time, speculation_time, speculation_accuracy, speculation_mode='integrated')

In [38]:
simulator._decoding_manager.get_finished_instruction_indices(simulator._window_manager.all_windows)

{1, 5, 9, 19, 23, 35, 44, 54}

In [43]:
simulator._window_manager.all_windows[7091]

Window((Region((4, 4), 6181, 7, 4, False, False),), frozenset(), frozenset({-1}), False)

In [32]:
ls_schedule.all_instructions[55]

Instruction(name='DISCARD', idx=55, patches=frozenset({(5, 8)}), duration=0, conditioned_on_idx=frozenset(), conditional_dependencies=frozenset(), conditioned_on_completion_idx=frozenset(), conditional_completion_dependencies=frozenset(), merge_faces=frozenset())

In [11]:
success, device_data, window_data, decoding_data = simulator.run(
    schedule=ls_schedule,
    scheduling_method='sliding',
    enforce_window_alignment=False,
    max_parallel_processes=None,
    progress_bar=True,
    save_animation_frames=False
)

Surface code rounds: 6100it [03:26, 15.06it/s] 

KeyboardInterrupt: 

Surface code rounds: 6100it [03:40, 15.06it/s]

In [11]:
plotter.plot_device_schedule_trace(device_data, hide_z_ticks=True)

<Axes3D: >